# Lab 9 Recommender Systems

## Due: Midnight, November 29th
In this lab, we’ll be working with the Book-Crossing, a book ratings data set to develop recommendation system algorithms, with the Surprise library. The Book-Crossing data is at http://www2.informatik.uni-freiburg.de/~cziegler/BX/ The Surprise is described at http://surpriselib.com


## Save Your Notebook! 
- Click on File (upper left corner), Select “Save” or press Ctrl+S.
- Important: You may loose your modification to a notebook if you do not Save it explicitly.
- Advice: Save often.  


## Submission
- Please follow the instructions and finish the exercises.
- After you finish the lab, please Click on File, Select “Download .ipynb”
- After download is complete, Click on File, Select “Print”, and and Choose ``Save as PDF''
- Submit both the Notebook file and the PDF File as your submission for Lab 9
- Please also submit the report for Lab 9.

# 1. Preparation


# 1.1. Install Packages
We need to isntall scikit-surprise, which is a Python scikit for building and analyzing recommender systems that deal with explicit rating data

In [1]:
# !pip install matplotlib
# !pip install networkx
!pip install scikit-surprise

In [2]:
import surprise
import pandas as pd

## 1.2 Load the data
Please download the BX-Book-Ratings.csv file from Canvas and upload to DS402 folder of Google Drive.

In [3]:
# The following code will mount the drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# read the csv file BX-Book-Ratings.csv
rating = pd.read_csv('/content/gdrive/My Drive/DS402/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'ISBN', 'bookRating']

Let's look at what the rating looks like

In [5]:
rating.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
print('# of rows in rating is: {}'.format(rating.shape[0]))

# of rows in rating is: 1149780


In [7]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   userID      1149780 non-null  int64 
 1   ISBN        1149780 non-null  object
 2   bookRating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## 1.3 Rating distribution analysis
We will analyze and visualize the rating distribution

In [8]:
# This funciton is used to setup the configuration for plotly
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [9]:
# Visualization of Rating Distribution
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = rating['bookRating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / rating.shape[0] * 100)], 
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
              )
# Create layout
layout = dict(title = 'Distribution Of {} book-ratings'.format(rating.shape[0]), 
              xaxis = dict(title = 'Rating'), 
              yaxis = dict(title = 'Count')
             )
# Create plot
configure_plotly_browser_state()
fig = go.Figure(data=[trace], layout=layout)
fig.show()

The rating scale is from 1 to 10, with 1 being the lowest rating and 10 being the highest rating. 0 means the rating is missing, i.e., user purchased the book but didn't leave a rating. There are around 62% of the ratings are missing, and very few ratings are 1 or 2, or 3, low rating books mean they are generally really bad. The 0 rating doesn't give us much infomration. Thus, we will remove these from the rating.

In [10]:
# remove 0 ratings
rating = rating[rating['bookRating'] > 0]

In [11]:
# now let's replot the distribution
# Visualization of Rating Distribution
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = rating['bookRating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / rating.shape[0] * 100)],
textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
              )
# Create layout
layout = dict(title = 'Distribution Of {} book-ratings'.format(rating.shape[0]), 
              xaxis = dict(title = 'Rating'), 
              yaxis = dict(title = 'Count')
             )
# Create plot
configure_plotly_browser_state()
fig = go.Figure(data=[trace], layout=layout)
fig.show()

Let's analyze the rating recevied by each book 


In [12]:
# Visualization of Ratings Distribution By Book
# Number of ratings per book
data = rating.groupby('ISBN')['bookRating'].count().clip(upper=50)
# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings', 
                     xbins = dict(start = 0, end = 50, size = 2)
                    )
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Book (Clipped at 50)', 
                   xaxis = dict(title = 'Number of Ratings Per Book'), 
                   yaxis = dict(title = 'Count'), bargap = 0.2
                  )
# Create plot
configure_plotly_browser_state()
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [13]:
rating.groupby('ISBN')['bookRating'].count().reset_index().sort_values('bookRating', ascending=False)[:10]

,ISBN,bookRating
26378,0316666343,707
132534,0971880107,581
44961,0385504209,487
22405,0312195516,383
90207,0679781587,333
5269,0060928336,320
77942,059035342X,313
15990,0142001740,307
58853,0446672211,295
54839,044023722X,281


We can observe that the histogram of the number of ratings received by books follows power-law distribution, i.e., the majority of the books only receive 1 rating, while only few books receive more than 10 ratings. The top 10 books that recieve most ratings are shown in the table above. For example, book ISBN 0316666343 received 707 ratings. 

In [14]:
# Visualization of Ratings Distribution By User
# Number of ratings per user
data = rating.groupby('userID')['bookRating'].count().clip(upper=50)
# Create trace
trace = go.Histogram(x = data.values, 
                     name = 'Ratings', 
                     xbins = dict(start = 0, end = 50, size = 2)
                    )
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 50)', 
                   xaxis = dict(title = 'Ratings Per User'), 
                   yaxis = dict(title = 'Count'), 
                   bargap = 0.2
                  )
# Create plot
configure_plotly_browser_state()
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [15]:
rating.groupby('userID')['bookRating'].count().reset_index().sort_values('bookRating', ascending=False)[:10]

,userID,bookRating
3160,11676,8524
27626,98391,5802
43027,153662,1969
52924,189835,1906
6510,23902,1395
21456,76499,1036
47780,171118,1035
65517,235105,1023
4555,16795,968
69413,248718,948


Similarly, the histogram of the number of ratings given by each user also follows power-law distribution, i.e., the majority of the user only give few ratings, while only few users give more than 20 ratings. The top 10 users who give the most ratings are shown in the table above. For example, user 11676 gives 8,524 ratings. 

## 1.3 Preprocessing on the data
To reduce the dimensionality of the data set, and avoid running into memory error, we will filter out rarely rated books and rarely rating users.

In [16]:
# find index of books that received larger than min_book_ratings
min_book_ratings = 15
filter_books = rating['ISBN'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

# find index of books who gave more than min_user_ratings
min_user_ratings = 15
filter_users = rating['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

# the new rating
rating_new = rating[(rating['ISBN'].isin(filter_books)) & (rating['userID'].isin(filter_users))]

In [17]:
rating.shape

(433671, 3)

In [18]:
rating_new.shape

(59503, 3)

## 1.3 Convert to Surprise Data Format
Now we are going to convert the data to suprise data format so we can call functions in surprise for training and evaluating recommender systems. To load a data set from the above pandas data frame, we will use the load_from_df() method. We will also need a Reader object, and the rating_scale parameter must be specified. The data frame must have three columns, corresponding to the user ids, the item ids, and the ratings in this order. Each row thus corresponds to a given rating. For more details of load_from_df(), please refer to https://surprise.readthedocs.io/en/stable/dataset.html#surprise.dataset.Dataset.load_from_df.

In [19]:
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NMF, BaselineOnly, SVD
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_new[['userID', 'ISBN', 'bookRating']], reader)

## 1.4 Split the Data into Training and Test Set
With the Surprise library, we will use several recommender systems to predict the missing ratings. Since we don't have groundtruth of the missing ratings, we will first split the data into training and test set with ratio as 4:1. We will train recommender systems on the training set. We then predict the rating on the test set and calculate the performance.

In [20]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=3)

# 2. Memory Based Collaborative Filtering
We will use surprise.prediction_algorithms.knns.KNNWithMeans(k=40, min_k=1, sim_options={}, verbose=True, \**kwargs) for memory based collaborative filtering. 

- k (int) – The (max) number of neighbors to take into account for aggregation. Default is 40.
- min_k (int) – The minimum number of neighbors to take into account for aggregation. If there are not enough neighbors, the neighbor aggregation is set to zero (so the prediction ends up being equivalent to the mean μu or μi). Default is 1.
- sim_options (dict) – A dictionary of options for the similarity measure. See Similarity measure configuration (https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration) for accepted options.
- verbose (bool) – Whether to print trace messages of bias estimation, similarity, etc. Default is True.

For more details, please refer to: https://surprise.readthedocs.io/en/stable/knn_inspired.html

## 2.1 User-based Collaborative Filtering

In [21]:
from surprise.prediction_algorithms.knns import KNNWithMeans

# specify the options for the algorithm
sim_options = {'name': 'cosine',  # we use cosine similarity as the measure to find k users
               'user_based': True  # compute  similarities between items, if we set user_based to True, then it computes similarities between users
               }
algo = KNNWithMeans(sim_options=sim_options)

# fit the model on the taining data and make prediction on the test set
predictions = algo.fit(trainset).test(testset)

# calculate the root mean square error (RMSE)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.6947


1.6947048064523094

In [22]:
# To inspect our predictions in details, we are going to build a pandas data frame with all the predictions.
def get_Iu(uid):
    """ return the number of items rated by given user
    args:
        uid: the id of the user
    returns: 
        the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(iid):
    """ return number of users that have rated given item
    args:
        iid: the raw id of the item
    returns:
        the number of users that have rated the item.
    """
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0

In [23]:
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [24]:
# get the good and bad prediction results
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [25]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
3442,208815,0842329250,10.0,10.0,"{'actual_k': 24, 'was_impossible': False}",9,27,0.0
2165,59172,0920668372,10.0,10.0,"{'actual_k': 3, 'was_impossible': False}",38,13,0.0
11329,203900,0385474016,10.0,10.0,"{'actual_k': 4, 'was_impossible': False}",4,30,0.0
11561,88229,0671021001,10.0,10.0,"{'actual_k': 17, 'was_impossible': False}",13,73,0.0
6582,68383,0618129022,10.0,10.0,"{'actual_k': 2, 'was_impossible': False}",6,16,0.0
9108,1211,0099771519,9.0,9.0,"{'actual_k': 0, 'was_impossible': False}",1,35,0.0
2177,241198,0060083948,7.0,7.0,"{'actual_k': 0, 'was_impossible': False}",1,6,0.0
2948,84479,0590353403,10.0,10.0,"{'actual_k': 40, 'was_impossible': False}",7,65,0.0
533,227836,0446310786,10.0,10.0,"{'actual_k': 12, 'was_impossible': False}",11,88,0.0
3633,243004,0140386335,10.0,10.0,"{'actual_k': 1, 'was_impossible': False}",3,14,0.0


In the above table,
- uid: user id
- iid: item id
- rui: actual rating from user u to item i
- est: predicted rating
- Iu: number of items rated by user u
- Ui: number of users that have rated item i
- err: error of predicted rating compared with actual rating

In [26]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
11752,239594,0446608386,9.0,1.299413,"{'actual_k': 1, 'was_impossible': False}",36,5,7.700587
3607,90049,0345361792,1.0,8.838657,"{'actual_k': 9, 'was_impossible': False}",9,69,7.838657
9280,22062,0440242002,1.0,9.000000,"{'actual_k': 0, 'was_impossible': False}",2,6,8.000000
1655,113279,0446364495,9.0,1.000000,"{'actual_k': 1, 'was_impossible': False}",5,6,8.000000
9992,45305,000649840X,10.0,2.000000,"{'actual_k': 0, 'was_impossible': False}",1,12,8.000000
8327,118333,0679886370,2.0,10.000000,"{'actual_k': 1, 'was_impossible': False}",2,7,8.000000
3083,643,1853260002,2.0,10.000000,"{'actual_k': 1, 'was_impossible': False}",6,9,8.000000
9085,155778,0525947647,1.0,9.332805,"{'actual_k': 10, 'was_impossible': False}",14,29,8.332805
1023,43910,0740704818,1.0,9.555758,"{'actual_k': 6, 'was_impossible': False}",42,14,8.555758
1364,256264,0312291639,1.0,10.000000,"{'actual_k': 2, 'was_impossible': False}",1,54,9.000000


## 2.2 Item-based Collaborative Filtering

## Exercise 1
Following Section 2.1, please apply item-based collaborative filtering and get the RMSE score. Please don't change the train and test split.

In [27]:
# TODO: please item-based collaborative filtering and get the accuracy as what we did in Section 2.1 
# Hint: for item-based collaborative filtering, we need to change the sim_options

from surprise.prediction_algorithms.knns import KNNWithMeans

# specify the options for the algorithm
sim_options = {'name': 'cosine',  # we use cosine similarity as the measure to find k users
               'user_based': False  # compute  similarities between items, if we set user_based to False, then it computes similarities between items
               }
algo = KNNWithMeans(sim_options=sim_options)

# fit the model on the taining data and make prediction on the test set
predictions = algo.fit(trainset).test(testset)

# calculate the root mean square error (RMSE)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.6507


1.650747105317031

Question: What's the RMSE score with item-based collaborative filtering?

Answer: 1.6507

## Exercise 2
Following Section 2.1, please show the worst 10 preidctions by the item-based collaborative filtering.

In [28]:
# To inspect our predictions in details, we are going to build a pandas data frame with all the predictions.
def get_Iu(uid):
    """ return the number of items rated by given user
    args:
        uid: the id of the user
    returns: 
        the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(iid):
    """ return number of users that have rated given item
    args:
        iid: the raw id of the item
    returns:
        the number of users that have rated the item.
    """
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0


In [29]:
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [30]:
# get the good and bad prediction results
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [31]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
6925,200323,059035342X,1.0,8.824702,"{'actual_k': 2, 'was_impossible': False}",2,133,7.824702
8327,118333,0679886370,2.0,10.000000,"{'actual_k': 1, 'was_impossible': False}",2,7,8.000000
4127,119027,0060083948,9.0,1.000000,"{'actual_k': 1, 'was_impossible': False}",3,6,8.000000
9465,97874,0743225082,10.0,1.980446,"{'actual_k': 2, 'was_impossible': False}",27,13,8.019554
1364,256264,0312291639,1.0,9.077441,"{'actual_k': 1, 'was_impossible': False}",1,54,8.077441
1398,114007,0385484518,1.0,9.128731,"{'actual_k': 4, 'was_impossible': False}",6,82,8.128731
1023,43910,0740704818,1.0,9.215422,"{'actual_k': 10, 'was_impossible': False}",42,14,8.215422
2672,219420,3257228007,1.0,9.428571,"{'actual_k': 0, 'was_impossible': False}",4,7,8.428571
9085,155778,0525947647,1.0,9.912390,"{'actual_k': 8, 'was_impossible': False}",14,29,8.912390
363,28619,1558506462,10.0,1.000000,"{'actual_k': 2, 'was_impossible': False}",15,5,9.000000


# 3 Model Based Collaborative Filtering
For model based collaborative filtering, we will mainly focus on matrix factorization based method. We evalaute two models, i.e., singular value decomposition (SVD) and nonnegative matrix factorization (NMF). We will use surprise.prediction_algorithms.matrix_factorization. For details of surprise.prediction_algorithms.matrix_factorization, please refer to: https://surprise.readthedocs.io/en/stable/matrix_factorization.html

## 3.1 Singular Value Decomposition
The objective function of singular value decomposition is very similar to what we have discussed for matrix factorization in class. We will use surprise.prediction_algorithms.matrix_factorization.SVD. The important parameters of the SVD function are:

Parameters:	
- n_factors – The number of factors. Default is 100.
- n_epochs – The number of iteration of the SGD procedure. Default is 20.
- biased (bool) – Whether to use baselines (or biases). See note above. Default is True.

For more details, please refer to https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

In [32]:
from surprise.prediction_algorithms.matrix_factorization import SVD
# specify the options for the algorithm
alg = SVD(n_factors=50, n_epochs=100, biased=True)

# train and predict ratings on the testset
predictions = alg.fit(trainset).test(testset)

# calculate the root mean square errot (RMSE)
accuracy.rmse(predictions)

RMSE: 1.6233


1.6233359138598267

## Exercise 3
The number of factors (we call latent dimension in the lecture) could affect the performance of SVD. Please vary n_factors as 10, 20, 50 and 100, and report the RMSE error below.

In [33]:
from surprise.prediction_algorithms.matrix_factorization import SVD
# specify the options for the algorithm
alg = SVD(n_factors=10, n_epochs=100, biased=True)

# train and predict ratings on the testset
predictions = alg.fit(trainset).test(testset)

# calculate the root mean square errot (RMSE)
accuracy.rmse(predictions)


RMSE: 1.7419


1.7418992116156113

In [34]:
from surprise.prediction_algorithms.matrix_factorization import SVD
# specify the options for the algorithm
alg = SVD(n_factors=20, n_epochs=100, biased=True)

# train and predict ratings on the testset
predictions = alg.fit(trainset).test(testset)

# calculate the root mean square errot (RMSE)
accuracy.rmse(predictions)

RMSE: 1.6948


1.6947660635020907

In [35]:
from surprise.prediction_algorithms.matrix_factorization import SVD
# specify the options for the algorithm
alg = SVD(n_factors=100, n_epochs=100, biased=True)

# train and predict ratings on the testset
predictions = alg.fit(trainset).test(testset)

# calculate the root mean square errot (RMSE)
accuracy.rmse(predictions)

RMSE: 1.5845


1.5844584918907763

TODO: Please fill in the RMSE error in the table below

| n_factors     |    10         |    20         |    50         |    100         |
| ------------- |:-------------:|:-------------:|:-------------:|:-------------:|
| RMSE          |    1.7419           |   1.6948            |   1.6233            |     1.5845          |

What pattern do you observe?

Answer: As the number of factors increased, the RMSE score decreased and it has better fit.

## 3.2 Non-negative Matrix Factorization
The non-negative matrix factorization is the same as matrix factorization. The only difference is non-negative matrix factorization has a constriant that the latent representations should be non-negative. This constriant is added becasue the ratings are usually non-negative. Such constraint can make sure that the predicted ratings is also non-negative. We will use the function surprise.prediction_algorithms.matrix_factorization.NMF. The main paramters of NMF are:

Parameters:	
- n_factors – The number of factors. Default is 15.
- n_epochs – The number of iteration of the SGD procedure. Default is 50.
- biased (bool) – Whether to use baselines (or biases). Default is False.

For more details, please refer to https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF

## Exercise 4
Please call surprise.prediction_algorithms.matrix_factorization.NMF with n_factors=50, n_epochs=100, and biased=True. Please calculate the mean square error

In [36]:
## TODO: please fill in your code here
from surprise.prediction_algorithms.matrix_factorization import NMF
# specify the options for the algorithm
alg = NMF(n_factors=50, n_epochs=100, biased=True)

# train and predict ratings on the testset
predictions = alg.fit(trainset).test(testset)

# calculate the root mean square errot (RMSE)
accuracy.rmse(predictions)

RMSE: 1.6260


1.6260465673898963